# Aktivitas partisipatif #3 (RandomForestClassifier)

Nama: Muhammad Karov Ardava Barus

NIM: 103052300001

Kelas: DS-47-01

## Import Libraries

Di tahap ini, kita akan memanggil libraries yang akan digunakan untuk membersihkan dataset yang kita dapatin

In [ ]:
import numpy as np
import pandas as pd # manipulasi data tabular
import pyswarms as ps
import warnings # warnings module
warnings.filterwarnings('ignore') # ignore message warning

from sklearn.preprocessing import MinMaxScaler, LabelEncoder # preprocessing data
from sklearn.model_selection import train_test_split, KFold # pemisahan data
from sklearn.naive_bayes import GaussianNB # model Naive Bayes
from sklearn.neural_network import MLPClassifier # model neural network
from sklearn.ensemble import RandomForestClassifier # model Random Forest
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score # evaluasi model

**a. Memanggil Data**

In [2]:
df = pd.read_excel('CAD alizadeh.xls') # memanggil data dengan format excel (.xls, .xlsx)

**b. Menampilkan data**

In [3]:
df.head(10) # menampilkan 10 data teratas

,No,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,1,53,90,175,Male,29.387755,0,1,1,0,...,4.7,141,5700,39,52,261,50,0,N,Cad
1,2,67,70,157,Fmale,28.398718,0,1,0,0,...,4.7,156,7700,38,55,165,40,4,N,Cad
2,3,54,54,164,Male,20.077335,0,0,1,0,...,4.7,139,7400,38,60,230,40,2,mild,Cad
3,4,66,67,158,Fmale,26.838648,0,1,0,0,...,4.4,142,13000,18,72,742,55,0,Severe,Normal
4,5,50,87,153,Fmale,37.165193,0,1,0,0,...,4.0,140,9200,55,39,274,50,0,Severe,Normal
5,6,50,75,175,Male,24.489796,0,0,1,0,...,4.2,141,7300,26,66,194,50,0,N,Cad
6,7,55,80,165,Male,29.384757,0,0,0,1,...,4.8,139,9400,58,33,292,40,4,mild,Cad
7,8,72,80,175,Male,26.122449,1,0,1,0,...,4.3,142,12200,25,74,410,45,4,mild,Cad
8,9,58,84,163,Fmale,31.615793,0,0,0,0,...,3.4,139,5100,49,50,370,50,0,N,Normal
9,10,60,71,170,Male,24.567474,1,0,0,0,...,4.6,140,4900,55,42,380,40,2,N,Cad


## Data Preprocessing

**c. Memeriksa apakah ada yang kosong pada data**

In [4]:
missing_values = df.isna().sum().sort_values(ascending=False) # mengecek missing value 

print('Total missing values tiap kolom:', '\n', missing_values) # menampilkan total missing value

Total missing values tiap kolom: 
 No                       0
Age                      0
Exertional CP            0
LowTH Ang                0
Q Wave                   0
St Elevation             0
St Depression            0
Tinversion               0
LVH                      0
Poor R Progression       0
FBS                      0
CR                       0
TG                       0
LDL                      0
HDL                      0
BUN                      0
ESR                      0
HB                       0
K                        0
Na                       0
WBC                      0
Lymph                    0
Neut                     0
PLT                      0
EF-TTE                   0
Region RWMA              0
VHD                      0
Nonanginal               0
Atypical                 0
Function Class           0
CVA                      0
Weight                   0
Length                   0
Sex                      0
BMI                      0
DM                  

In [5]:
missing_values = df.isna().sum().sum() # mengecek total missing value 

print(f'Total missing values: {missing_values}') # menampilkan total missing value

Total missing values: 0


In [6]:
print(f'Total data duplikat: {df.duplicated().sum()}') # mengecek total data duplikat

Total data duplikat: 0


**d. Mengubah semua kolom menjadi bernilai numerik**

Untuk mengubah data kategorik menjadi data numerik, kita akan menggunakan metode `Label Encoding`

In [7]:
cat_cols = df.select_dtypes(include='object').columns # mengambil kolom kategorikal
num_cols = df.select_dtypes(include='number').columns # mengambil kolom numerik

In [8]:
le = LabelEncoder() # inisialisasi label encoder

for col in cat_cols: # loop untuk setiap kolom kategorikal
    df[col] = le.fit_transform(df[col]) # transformasi data kategorikal menjadi numerik

df.head(10) # menampilkan 10 data teratas

,No,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,1,53,90,175,1,29.387755,0,1,1,0,...,4.7,141,5700,39,52,261,50,0,1,0
1,2,67,70,157,0,28.398718,0,1,0,0,...,4.7,156,7700,38,55,165,40,4,1,0
2,3,54,54,164,1,20.077335,0,0,1,0,...,4.7,139,7400,38,60,230,40,2,3,0
3,4,66,67,158,0,26.838648,0,1,0,0,...,4.4,142,13000,18,72,742,55,0,2,1
4,5,50,87,153,0,37.165193,0,1,0,0,...,4.0,140,9200,55,39,274,50,0,2,1
5,6,50,75,175,1,24.489796,0,0,1,0,...,4.2,141,7300,26,66,194,50,0,1,0
6,7,55,80,165,1,29.384757,0,0,0,1,...,4.8,139,9400,58,33,292,40,4,3,0
7,8,72,80,175,1,26.122449,1,0,1,0,...,4.3,142,12200,25,74,410,45,4,3,0
8,9,58,84,163,0,31.615793,0,0,0,0,...,3.4,139,5100,49,50,370,50,0,1,1
9,10,60,71,170,1,24.567474,1,0,0,0,...,4.6,140,4900,55,42,380,40,2,1,0


**e. Melakukan proses normalisasi pada masing-masing atribut**

Normalisasi biasanya diterapkan pada data numerik untuk memastikan bahwa semua fitur berada dalam skala yang sama, tetapi **tidak perlu** dinormalisasiin pada data kategorikal. Metode yang kita gunakan adalah `Min-Max Scaling`

In [9]:
scaler = MinMaxScaler() # inisialisasi minmax scaler

df[num_cols] = scaler.fit_transform(df[num_cols]) # transformasi data numerik menggunakan minmax scaler

df.head(10) # menampilkan 10 data teratas

,No,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,0.000000,0.410714,0.583333,0.729167,1,0.494721,0.0,1.0,1.0,0.0,...,0.472222,0.464286,0.139860,0.603774,0.350877,0.329149,0.777778,0.0,1,0
1,0.003311,0.660714,0.305556,0.354167,0,0.451314,0.0,1.0,0.0,0.0,...,0.472222,1.000000,0.279720,0.584906,0.403509,0.195258,0.555556,1.0,1,0
2,0.006623,0.428571,0.083333,0.500000,1,0.086105,0.0,0.0,1.0,0.0,...,0.472222,0.392857,0.258741,0.584906,0.491228,0.285914,0.555556,0.5,3,0
3,0.009934,0.642857,0.263889,0.375000,0,0.382846,0.0,1.0,0.0,0.0,...,0.388889,0.500000,0.650350,0.207547,0.701754,1.000000,0.888889,0.0,2,1
4,0.013245,0.357143,0.541667,0.270833,0,0.836058,0.0,1.0,0.0,0.0,...,0.277778,0.428571,0.384615,0.905660,0.122807,0.347280,0.777778,0.0,2,1
5,0.016556,0.357143,0.375000,0.729167,1,0.279759,0.0,0.0,1.0,0.0,...,0.333333,0.464286,0.251748,0.358491,0.596491,0.235704,0.777778,0.0,1,0
6,0.019868,0.446429,0.444444,0.520833,1,0.494590,0.0,0.0,0.0,1.0,...,0.500000,0.392857,0.398601,0.962264,0.017544,0.372385,0.555556,1.0,3,0
7,0.023179,0.750000,0.444444,0.729167,1,0.351413,1.0,0.0,1.0,0.0,...,0.361111,0.500000,0.594406,0.339623,0.736842,0.536960,0.666667,1.0,3,0
8,0.026490,0.500000,0.500000,0.479167,0,0.592505,0.0,0.0,0.0,0.0,...,0.111111,0.392857,0.097902,0.792453,0.315789,0.481172,0.777778,0.0,1,1
9,0.029801,0.535714,0.319444,0.625000,1,0.283168,1.0,0.0,0.0,0.0,...,0.444444,0.428571,0.083916,0.905660,0.175439,0.495119,0.555556,0.5,1,0


**⚠️ Tambahan: Di feature `No` sebenarnya tidak bermakna, karena hanya sebagai index, contoh: (1, 2, 3, ..., n), sehingga disarankan untuk dihapus saja.**

In [10]:
df.drop(columns=['No'], inplace=True) # menghapus kolom No

df.head(10) # menampilkan 10 data teratas

,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,FH,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,0.410714,0.583333,0.729167,1,0.494721,0.0,1.0,1.0,0.0,0.0,...,0.472222,0.464286,0.139860,0.603774,0.350877,0.329149,0.777778,0.0,1,0
1,0.660714,0.305556,0.354167,0,0.451314,0.0,1.0,0.0,0.0,0.0,...,0.472222,1.000000,0.279720,0.584906,0.403509,0.195258,0.555556,1.0,1,0
2,0.428571,0.083333,0.500000,1,0.086105,0.0,0.0,1.0,0.0,0.0,...,0.472222,0.392857,0.258741,0.584906,0.491228,0.285914,0.555556,0.5,3,0
3,0.642857,0.263889,0.375000,0,0.382846,0.0,1.0,0.0,0.0,0.0,...,0.388889,0.500000,0.650350,0.207547,0.701754,1.000000,0.888889,0.0,2,1
4,0.357143,0.541667,0.270833,0,0.836058,0.0,1.0,0.0,0.0,0.0,...,0.277778,0.428571,0.384615,0.905660,0.122807,0.347280,0.777778,0.0,2,1
5,0.357143,0.375000,0.729167,1,0.279759,0.0,0.0,1.0,0.0,0.0,...,0.333333,0.464286,0.251748,0.358491,0.596491,0.235704,0.777778,0.0,1,0
6,0.446429,0.444444,0.520833,1,0.494590,0.0,0.0,0.0,1.0,0.0,...,0.500000,0.392857,0.398601,0.962264,0.017544,0.372385,0.555556,1.0,3,0
7,0.750000,0.444444,0.729167,1,0.351413,1.0,0.0,1.0,0.0,0.0,...,0.361111,0.500000,0.594406,0.339623,0.736842,0.536960,0.666667,1.0,3,0
8,0.500000,0.500000,0.479167,0,0.592505,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.392857,0.097902,0.792453,0.315789,0.481172,0.777778,0.0,1,1
9,0.535714,0.319444,0.625000,1,0.283168,1.0,0.0,0.0,0.0,0.0,...,0.444444,0.428571,0.083916,0.905660,0.175439,0.495119,0.555556,0.5,1,0


Data kita sudah bersih!

## Modelling
**Split data menjadi training dan test set**


Kita akan split data train dan test menjadi 80:20

In [11]:
# membagi data menjadi data train dan data test
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['Cath']), # features
    df['Cath'], # target (yang ingin diprediksi)
    test_size=0.3, # ukuran data test 20% dari total data
    random_state=42 # seed agar hasil random tetap konsisten
)

**Pemodelan dengan Naive Bayes**

In [12]:
model_nb = GaussianNB() # inisialisasi model Naive Bayes
model_nb.fit(X_train, y_train) # training model

y_pred_nb = model_nb.predict(X_test) # prediksi data test

print(f'Akurasi model: {accuracy_score(y_test, y_pred_nb)}') # evaluasi model menggunakan accuracy score
print(f'F1 score model: {f1_score(y_test, y_pred_nb)}') # evaluasi model menggunakan f1 score
print(f'Confusion matrix model: \n{confusion_matrix(y_test, y_pred_nb)}') # evaluasi model menggunakan confusion matrix

Akurasi model: 0.4945054945054945
F1 score model: 0.5208333333333334
Confusion matrix model: 
[[20 46]
 [ 0 25]]


**Pemodelan dengan Random Forest**

In [13]:
model_rf = RandomForestClassifier(n_estimators=100, random_state=42) # inisialisasi model Random Forest
model_rf.fit(X_train, y_train) # training model

y_pred_rf = model_rf.predict(X_test) # prediksi data test

print(f'Akurasi model: {accuracy_score(y_test, y_pred_rf)}') # evaluasi model menggunakan accuracy score
print(f'F1 score model: {f1_score(y_test, y_pred_rf)}') # evaluasi model menggunakan f1 score
print(f'Confusion matrix model: \n{confusion_matrix(y_test, y_pred_rf)}') # evaluasi model menggunakan confusion matrix

Akurasi model: 0.8571428571428571
F1 score model: 0.6976744186046512
Confusion matrix model: 
[[63  3]
 [10 15]]


**Pemodelan dengan Multi-layer Perceptron dengan optimasi Particle Swarm Optimization (PSO)**

In [14]:
# !pip install pyswarms

In [ ]:
# fungsi objective untuk PSO (Particle Swarm Optimization)
def objective_function(params):
    # extract parameters
    n_hidden_neurons = params[:, 0].astype(int)
    alpha = params[:, 1]
    learning_rate_init = params[:, 2]
    beta_1 = params[:, 3]
    beta_2 = params[:, 4]
    
    n_particles = params.shape[0]
    scores = np.zeros(n_particles)
    
    # 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for i, params_i in enumerate(params):
        cv_scores = []

        # melakukan cross-validation menggunakan KFold dengan 5 fold
        for train_idx, test_idx in kf.split(X_train):
            X_cv_train, X_cv_test = X_train.iloc[train_idx], X_train.iloc[test_idx]
            y_cv_train, y_cv_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
            
            # melakukan pelatihan model
            model = MLPClassifier(
                hidden_layer_sizes=(int(params_i[0]),),     
                alpha=params_i[1], # regularization parameter
                learning_rate_init=params_i[2],
                beta_1=params_i[3], # momentum parameter
                beta_2=params_i[4], # momentum parameter
                max_iter=300, # maximum number of iterations
                random_state=42,
                activation='relu', # fungsi aktivasi
                solver='adam' # solver untuk optimasi
            )
            model.fit(X_cv_train, y_cv_train)
            
            # evaluasi model
            y_pred = model.predict(X_cv_test)
            acc = accuracy_score(y_cv_test, y_pred)
            cv_scores.append(acc)
        
        # hitung mean accuracy dan meniadakan masalah minimalisasi
        scores[i] = -np.mean(cv_scores)
    
    return scores

#########
# hidden_neurons: 10-100
# alpha: 0.0001-0.01
# learning_rate_init: 0.001-0.1
# beta_1: 0.8-0.999
# beta_2: 0.9-0.9999
#########

bounds = (np.array([10, 0.0001, 0.001, 0.8, 0.9]), 
          np.array([100, 0.01, 0.1, 0.999, 0.9999]))

# inisialisasi swarm dengan 20 particles
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 3, 'p': 2} # parameter PSO
optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=5, options=options, bounds=bounds)

# optimasi hyperparameters
print("Starting PSO optimization - this may take a few minutes...")
cost, best_params = optimizer.optimize(objective_function, iters=10)

# mendapatkan best parameters 
best_hidden_neurons = int(best_params[0])
best_alpha = best_params[1]
best_learning_rate = best_params[2]
best_beta_1 = best_params[3]
best_beta_2 = best_params[4]

print("Best parameters found:")
print(f"Hidden neurons: {best_hidden_neurons}")
print(f"Alpha (regularization): {best_alpha:.8f}")
print(f"Learning rate: {best_learning_rate:.6f}")
print(f"Beta 1: {best_beta_1:.6f}")
print(f"Beta 2: {best_beta_2:.6f}")

# train model MLP dengan parameter hasil PSO
model_mlp = MLPClassifier(
    hidden_layer_sizes=(best_hidden_neurons,),
    alpha=best_alpha,
    learning_rate_init=best_learning_rate,
    beta_1=best_beta_1,
    beta_2=best_beta_2,
    max_iter=500,
    random_state=42,
    activation='relu',
    solver='adam'
)
model_mlp.fit(X_train, y_train)

# evaluasi model MLP di data test
y_pred_mlp = model_mlp.predict(X_test)
mlp_accuracy = accuracy_score(y_test, y_pred_mlp)
mlp_f1 = f1_score(y_test, y_pred_mlp)
mlp_cm = confusion_matrix(y_test, y_pred_mlp)

print(f"\nFinal MLP Classifier performance:")
print(f"Accuracy: {mlp_accuracy:.4f}")
print(f"F1 Score: {mlp_f1:.4f}")
print("Confusion Matrix:")
print(mlp_cm)

# bandingin dengan previous models
print("\nModel Comparison:")
print(f"Naive Bayes Accuracy: {accuracy_score(y_test, y_pred_nb):.4f}")
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Optimized MLP Accuracy: {mlp_accuracy:.4f}")

2025-05-09 16:20:42,628 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 3, 'p': 2}


Starting PSO optimization - this may take a few minutes...


pyswarms.single.global_best: 100%|██████████|10/10, best_cost=-0.845
2025-05-09 16:22:30,932 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.8447397563676633, best pos: [8.72341252e+01 8.48331068e-03 4.26945770e-02 8.46971047e-01
 9.80373348e-01]


Best parameters found:
Hidden neurons: 87
Alpha (regularization): 0.00848331
Learning rate: 0.042695
Beta 1: 0.846971
Beta 2: 0.980373

Final MLP Classifier performance:
Accuracy: 0.8681
F1 Score: 0.7500
Confusion Matrix:
[[61  5]
 [ 7 18]]

Model Comparison:
Naive Bayes Accuracy: 0.4945
Random Forest Accuracy: 0.8571
Optimized MLP Accuracy: 0.8681


**Hasil Analisis Perbandingan Model**

Dari hasil pengujian ketiga model machine learning (Neural Network dengan PSO, Random Forest, dan Naive Bayes), kita dapat melihat beberapa temuan menarik:

### 1. Neural Network dengan Optimasi PSO
Neural Network menunjukkan performa terbaik dengan karakteristik:
- **Akurasi**: 86.81% (tertinggi dari semua model)
- **F1-Score**: 75.00% (tertinggi dari semua model)
- **Parameter Optimal PSO**:
  ```python
  Hidden neurons: 87
  Alpha: 0.00848331
  Learning rate: 0.042695
  Beta 1: 0.846971
  Beta 2: 0.980373
  ```
- **Confusion Matrix**:
  ```python
  [[61  5]
   [ 7 18]]
  ```
  - True Negative: 61 kasus
  - False Positive: 5 kasus
  - False Negative: 7 kasus
  - True Positive: 18 kasus

### 2. Random Forest
- **Akurasi**: 85.71%
- **F1 score**: 69.76%
- Performa sedikit di bawah Neural Network
- Menunjukkan kemampuan yang baik dalam menangani data non-linear
- False positive dan false negative yang seimbang
- **Confusion Matrix**:
    ```python
    [[63  3]
    [10 15]]
    ```
    - True Negative: 63 kasus
    - False Positive: 3 kasus
    - False Negative: 10 kasus
    - True Positive: 15 kasus

### 3. Naive Bayes
- **Akurasi**: 49.45%
- **F1 score**: 52.08%
- Performa paling rendah di antara ketiga model
- Menunjukkan ketidakseimbangan dalam prediksi
- Asumsi independensi antar fitur tidak terpenuhi
- **Confusion Matrix**:
    ```python
    [[20 46]
    [ 0 25]]
    ```
    - True Negative: 20 kasus
    - False Positive: 46 kasus
    - False Negative: 0 kasus
    - True Positive: 25 kasus

### Kesimpulan
Neural Network dengan optimasi PSO menjadi pilihan terbaik untuk dataset ini karena:
1. Memberikan peningkatan akurasi 1.1% dari Random Forest dan 37.36% dari Naive Bayes
2. Menunjukkan keseimbangan yang baik dalam prediksi (dilihat dari confusion matrix)
3. Mampu menangkap pola kompleks berkat arsitektur deep learning
4. Optimasi PSO berhasil menemukan parameter yang optimal untuk model

Hasil ini membuktikan bahwa kompleksitas tambahan dari Neural Network dan proses optimasi PSO memberikan manfaat nyata dalam peningkatan akurasi model.